In [442]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import RobustScaler , MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, LSTM
from skimage.metrics import mean_squared_error
import math
import plotly.graph_objects as go

In [443]:
original_crime_df = pd.read_csv("Crime_Data_from_2020_to_Present.csv")
print(original_crime_df.head())
print(original_crime_df['Crm Cd'].value_counts()[:10])
new_dataset = original_crime_df[original_crime_df["Crm Cd"].isin([510,624,740,330,310,230,626,440,420,354]) ]
new_dataset.head()

       DR_NO               Date Rptd                DATE OCC  TIME OCC  AREA  \
0   10304468  01/08/2020 12:00:00 AM  01/08/2020 12:00:00 AM      2230     3   
1  190101086  01/02/2020 12:00:00 AM  01/01/2020 12:00:00 AM       330     1   
2  201220752  09/16/2020 12:00:00 AM  09/16/2020 12:00:00 AM      1230    12   
3  191501505  01/01/2020 12:00:00 AM  01/01/2020 12:00:00 AM      1730    15   
4  191921269  01/01/2020 12:00:00 AM  01/01/2020 12:00:00 AM       415    19   

     AREA NAME  Rpt Dist No  Part 1-2  Crm Cd  \
0    Southwest          377         2     624   
1      Central          163         2     624   
2  77th Street         1259         2     745   
3  N Hollywood         1543         2     745   
4      Mission         1998         2     740   

                                         Crm Cd Desc  ... Status  Status Desc  \
0                           BATTERY - SIMPLE ASSAULT  ...     AO  Adult Other   
1                           BATTERY - SIMPLE ASSAULT  ...     

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468
10,200100509,01/04/2020 12:00:00 AM,01/04/2020 12:00:00 AM,2200,1,Central,192,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,NaN,NaN,NaN,15TH,OLIVE,34.0359,-118.2648
15,201811813,05/28/2020 12:00:00 AM,05/25/2020 12:00:00 AM,200,18,Southeast,1851,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,MENLO AV,130TH ST,33.9144,-118.2894


In [444]:
dataset_510 = original_crime_df[original_crime_df["Crm Cd"].isin([510]) ]
dataset_624 = original_crime_df[original_crime_df["Crm Cd"].isin([624])]
dataset_740 = original_crime_df[original_crime_df["Crm Cd"].isin([740])]
dataset_330 = original_crime_df[original_crime_df["Crm Cd"].isin([330])]
dataset_310 = original_crime_df[original_crime_df["Crm Cd"].isin([310])]
dataset_230 = original_crime_df[original_crime_df["Crm Cd"].isin([230])]
dataset_626 = original_crime_df[original_crime_df["Crm Cd"].isin([626])]
dataset_440 = original_crime_df[original_crime_df["Crm Cd"].isin([440])]
dataset_420 = original_crime_df[original_crime_df["Crm Cd"].isin([420])]
dataset_354 = original_crime_df[original_crime_df["Crm Cd"].isin([354])]

dataset_510

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
15,201811813,05/28/2020 12:00:00 AM,05/25/2020 12:00:00 AM,200,18,Southeast,1851,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,MENLO AV,130TH ST,33.9144,-118.2894
24,201713253,09/15/2020 12:00:00 AM,09/14/2020 12:00:00 AM,1305,17,Devonshire,1776,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,16700 LABRADOR ST,NaN,34.2493,-118.4965
27,200505212,01/29/2020 12:00:00 AM,01/26/2020 12:00:00 AM,1200,5,Harbor,529,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,E,BROAD,33.7753,-118.2456
32,200117988,09/15/2020 12:00:00 AM,09/03/2020 12:00:00 AM,2000,1,Central,111,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,500 N FIGUEROA ST,NaN,34.0615,-118.2470
50,201312607,06/21/2020 12:00:00 AM,06/21/2020 12:00:00 AM,30,13,Newton,1321,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,100 E 24TH ST,NaN,34.0266,-118.2676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317802,210911274,07/05/2021 12:00:00 AM,07/03/2021 12:00:00 AM,1900,9,Van Nuys,911,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,15600 VANOWEN ST,NaN,34.1939,-118.4732
317808,211204656,01/11/2021 12:00:00 AM,01/10/2021 12:00:00 AM,2300,12,77th Street,1259,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,400 E 74TH ST,NaN,33.9730,-118.2657
317846,210404776,01/21/2021 12:00:00 AM,01/19/2021 12:00:00 AM,1600,4,Hollenbeck,497,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,1400 S DE LA TORRE WY,NaN,34.0166,-118.2058
317849,211208872,03/19/2021 12:00:00 AM,03/19/2021 12:00:00 AM,1105,12,77th Street,1218,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,58TH ST,FIGUEROA ST,33.9897,-118.2827


In [445]:
dataset_510['Date Rptd'] = pd.to_datetime(dataset_510['Date Rptd'])
dataset_510['DATE OCC'] = pd.to_datetime(dataset_510['DATE OCC'])

dataset_510.sort_values(by="DATE OCC")

/var/folders/8g/z0__2mts0rnbjh3kq8zsfz2w0000gn/T/ipykernel_6830/214316822.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/8g/z0__2mts0rnbjh3kq8zsfz2w0000gn/T/ipykernel_6830/214316822.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
15061,201604004,2020-01-01,2020-01-01,5,16,Foothill,1601,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,11900 COMETA AV,NaN,34.2866,-118.4125
9729,201104043,2020-01-02,2020-01-01,1830,11,Northeast,1145,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,2500 GLENDALE BL,NaN,34.1027,-118.2592
9728,201104038,2020-01-01,2020-01-01,800,11,Northeast,1149,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,100 S AVENUE 57,NaN,34.1074,-118.1978
1434,200204057,2020-01-02,2020-01-01,2000,2,Rampart,204,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,3400 PLATA ST,NaN,34.0777,-118.2821
1432,200204055,2020-01-02,2020-01-01,2200,2,Rampart,275,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,BURLINGTON,JAMES M WOOD,34.0519,-118.2755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292740,211011359,2021-08-16,2021-08-16,705,10,West Valley,1049,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,6300 BALBOA BL,NaN,0.0000,0.0000
317434,211815272,2021-08-16,2021-08-16,400,18,Southeast,1823,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,10100 S SAN PEDRO ST,NaN,33.9438,-118.2695
201876,210115282,2021-08-16,2021-08-16,1900,1,Central,138,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,BOYD ST,WALL ST,0.0000,0.0000
277674,211711552,2021-08-16,2021-08-16,510,17,Devonshire,1794,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,18500 PARTHENIA ST,NaN,34.2282,-118.5367


In [446]:
df1 = dataset_510['DATE OCC'].dt.date.value_counts().sort_index().reset_index()
df1.columns = ['DATE','Count']

df1['DATE'] = pd.to_datetime(df1['DATE']) - pd.to_timedelta(7, unit='d')
df1= df1.groupby([pd.Grouper(key='DATE', freq='W')])['Count'].sum()

df1 = pd.DataFrame(data=df1.values, columns=['Count'])
df1 = df1.values
df1 = df1.astype('int32')
df1

array([[194],
       [344],
       [308],
       [317],
       [342],
       [312],
       [310],
       [306],
       [306],
       [322],
       [336],
       [307],
       [361],
       [412],
       [415],
       [412],
       [442],
       [394],
       [450],
       [413],
       [436],
       [400],
       [459],
       [468],
       [495],
       [514],
       [522],
       [431],
       [438],
       [438],
       [442],
       [414],
       [421],
       [382],
       [429],
       [384],
       [374],
       [388],
       [372],
       [387],
       [415],
       [382],
       [438],
       [411],
       [410],
       [409],
       [423],
       [397],
       [414],
       [403],
       [391],
       [385],
       [416],
       [400],
       [407],
       [442],
       [433],
       [484],
       [458],
       [466],
       [411],
       [435],
       [408],
       [391],
       [393],
       [408],
       [386],
       [398],
       [441],
       [460],
       [405],
      

In [447]:
def formatNumber(n):
    return '{:,}'.format(n).replace(","," ")
def highlight_diag(df):
    a = np.full(df.shape, '', dtype='<U24')
    np.fill_diagonal(a, 'background-color: yellow')
    return pd.DataFrame(a, index=df.index, columns=df.columns)
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [448]:
list_graphs = []
graph_1 = go.Scatter(x=list(range(0,len(df1))),y=df1.flatten(),name="Dataset")
list_graphs.append(graph_1)
average = moving_average(df1)
graph_2 = go.Scatter(x=list(range(0,len(df1))),y=average,name="Average")
list_graphs.append(graph_2)
fig = go.Figure(data=list_graphs)
fig.update_layout(width=1024,height=500,title="Dataset of crimes in ",xaxis=dict(title='Weeks'),yaxis=dict(title='Number of crimes'))

fig.show()

In [449]:
scaler = MinMaxScaler()
df1 = scaler.fit_transform(df1)

In [450]:
# dataset_510['DATE OCC'] = pd.to_datetime(dataset_510['DATE OCC']) - pd.to_timedelta(7, unit='d')

df1
train_size = int(len(df1) * 0.7)
print(train_size)
test_size = len(df1) - train_size
print(test_size)
train, test = df1[0:train_size,:], df1[train_size:len(df1),:]

# df1 = df1.drop(['DATE'], axis=1, index=None, columns=None, level=None, inplace=False, errors='raise')\

60
26


In [451]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back),0]
        dataX.append(a)
        dataY.append(dataset[i + look_back,0])
    return np.array(dataX), np.array(dataY)

In [452]:
# I make my dataset
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [453]:
trainX.shape, trainY.shape, testX.shape, testY.shape

((59, 1), (59,), (25, 1), (25,))

In [454]:
trainX = np.reshape(trainX, (trainX.shape[0],1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0],1, testX.shape[1]))

In [455]:
model = Sequential()
model.add(LSTM(8,activation='tanh',return_sequences=True, input_shape=(1,look_back)))
model.add(LSTM(4,activation='tanh',return_sequences=False, input_shape=(1,look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.fit(trainX, trainY, epochs=500, batch_size=4, verbose=1)

Epoch 1/500
15/15 [==============================] - 2s 2ms/step - loss: 0.5503
Epoch 2/500
15/15 [==============================] - 0s 1ms/step - loss: 0.4939
Epoch 3/500
15/15 [==============================] - 0s 1ms/step - loss: 0.4438
Epoch 4/500
15/15 [==============================] - 0s 2ms/step - loss: 0.3946
Epoch 5/500
15/15 [==============================] - 0s 1ms/step - loss: 0.3459
Epoch 6/500
15/15 [==============================] - 0s 1ms/step - loss: 0.2976
Epoch 7/500
15/15 [==============================] - 0s 1ms/step - loss: 0.2496
Epoch 8/500
15/15 [==============================] - 0s 1ms/step - loss: 0.2024
Epoch 9/500
15/15 [==============================] - 0s 1ms/step - loss: 0.1580
Epoch 10/500
15/15 [==============================] - 0s 1ms/step - loss: 0.1174
Epoch 11/500
15/15 [==============================] - 0s 1ms/step - loss: 0.0819
Epoch 12/500
15/15 [==============================] - 0s 1ms/step - loss: 0.0521
Epoch 13/500
15/15 [=================

In [456]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

1/1 [==============================] - 0s 13ms/step


In [457]:
trainPredict.shape, trainY.shape
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [458]:
trainY[0],trainPredict[:,0]

(array([344., 308., 317., 342., 312., 310., 306., 306., 322., 336., 307.,
        361., 412., 415., 412., 442., 394., 450., 413., 436., 400., 459.,
        468., 495., 514., 522., 431., 438., 438., 442., 414., 421., 382.,
        429., 384., 374., 388., 372., 387., 415., 382., 438., 411., 410.,
        409., 423., 397., 414., 403., 391., 385., 416., 400., 407., 442.,
        433., 484., 458., 466.]),
 array([256.67755, 355.12228, 330.01862, 336.22653, 353.7098 , 332.7716 ,
        331.3939 , 328.6458 , 328.6458 , 339.69604, 349.48404, 329.3319 ,
        367.20013, 404.0251 , 406.21014, 404.0251 , 425.9213 , 390.94925,
        431.7694 , 404.75327, 421.53622, 395.30038, 438.348  , 444.92303,
        464.5963 , 478.36224, 484.1317 , 417.8834 , 422.99768, 422.99768,
        425.9213 , 405.48163, 410.58417, 382.27405, 416.42282, 383.71716,
        376.51385, 386.6068 , 375.07715, 385.88394, 406.21014, 382.27405,
        422.99768, 403.297  , 402.5691 , 401.84143, 412.0432 , 393.12378,
    

In [459]:
testY[0],testPredict[:,0]

(array([435., 408., 391., 393., 408., 386., 398., 441., 460., 405., 405.,
        405., 433., 405., 422., 415., 412., 433., 410., 368., 390., 429.,
        367., 382.,  23.]),
 array([403.297  , 420.80554, 401.1139 , 388.7769 , 390.22485, 401.1139 ,
        385.16147, 393.8491 , 425.19034, 439.07883, 398.93237, 398.93237,
        398.93237, 419.34433, 398.93237, 411.31366, 406.21014, 404.0251 ,
        419.34433, 402.5691 , 372.20764, 388.05325, 416.42282, 371.49115,
        382.27405], dtype=float32))

In [460]:
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 32.09 RMSE
Test Score: 75.99 RMSE


In [461]:
trainPredictPlot = np.empty_like(df1)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

In [462]:
testPredictPlot = np.empty_like(df1)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back):len(df1)-1, :] = testPredict

In [485]:
listGraph = []
listGraph.append(go.Scatter(x=list(range(0,len(df1))),y=scaler.inverse_transform(df1).flatten(),name="Original dataset"))
listGraph.append(go.Scatter(x=list(range(0,len(trainPredictPlot))),y=trainPredictPlot.flatten(),name="Predict with Train set"))
listGraph.append(go.Scatter(x=list(range(0,len(testPredictPlot))),y=testPredictPlot.flatten(),name="Predict with Test set"))

fig = go.Figure(data=listGraph)
fig.update_layout(width=1024,height=500,title="[LSTM] Prediction for Stolen Vehicle (Code 510)",xaxis=dict(title='Weeks'),yaxis=dict(title='Number of crimes'))

fig.show()

In [464]:
dataset_624

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
16,201707577,03/20/2020 12:00:00 AM,03/20/2020 12:00:00 AM,1320,17,Devonshire,1752,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,20900 DEVONSHIRE ST,NaN,34.2572,-118.5885
68,200104054,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,2330,1,Central,135,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,WALL,5TH ST,34.0495,-118.2458
70,200104058,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,1940,1,Central,135,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,LOS ANGELES ST,2ND ST,34.0485,-118.2447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317819,210407038,03/25/2021 12:00:00 AM,03/25/2021 12:00:00 AM,1015,4,Hollenbeck,497,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,3400 EMERY ST,NaN,34.0153,-118.2061
317838,210206530,03/05/2021 12:00:00 AM,03/05/2021 12:00:00 AM,915,2,Rampart,246,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,600 S ALVARADO ST,NaN,34.0593,-118.2750
317842,210610209,05/25/2021 12:00:00 AM,05/25/2021 12:00:00 AM,813,6,Hollywood,645,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,6800 HOLLYWOOD BL,NaN,34.1016,-118.3391
317848,212110947,07/04/2021 12:00:00 AM,07/04/2021 12:00:00 AM,2135,21,Topanga,2143,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,23100 FRIAR ST,NaN,34.1855,-118.6296


In [465]:
dataset_624['Date Rptd'] = pd.to_datetime(dataset_624['Date Rptd'])
dataset_624['DATE OCC'] = pd.to_datetime(dataset_624['DATE OCC'])

dataset_624.sort_values(by="DATE OCC")

/var/folders/8g/z0__2mts0rnbjh3kq8zsfz2w0000gn/T/ipykernel_6830/2874154411.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/8g/z0__2mts0rnbjh3kq8zsfz2w0000gn/T/ipykernel_6830/2874154411.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
4310,200504040,2020-01-01,2020-01-01,1250,5,Harbor,589,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,3700 STEPHEN M WHITE DR,NaN,33.7105,-118.2860
13022,201404017,2020-01-01,2020-01-01,330,14,Pacific,1436,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,11200 WESTMINSTER AV,NaN,34.0172,-118.4230
13021,201404016,2020-01-01,2020-01-01,330,14,Pacific,1436,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,11200 WESTMINSTER AV,NaN,34.0172,-118.4230
12831,201305897,2020-02-03,2020-01-01,700,13,Newton,1367,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,5100 HOLMES AV,NaN,33.9962,-118.2411
179578,201104029,2020-01-01,2020-01-01,930,11,Northeast,1102,2,624,BATTERY - SIMPLE ASSAULT,...,AA,Adult Arrest,624.0,NaN,NaN,NaN,4000 PERLITA AV,NaN,34.1254,-118.2648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230438,210614359,2021-08-16,2021-08-16,900,6,Hollywood,646,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,SELMA,CASSIL,34.0998,-118.3338
276600,211416681,2021-08-16,2021-08-16,935,14,Pacific,1402,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,SAWTELLE BL,NATIONAL BL,34.0266,-118.4317
226343,211513215,2021-08-16,2021-08-16,1534,15,N Hollywood,1566,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,4400 LANKERSHIM BL,NaN,0.0000,0.0000
201090,210115277,2021-08-16,2021-08-16,1110,1,Central,166,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,400 E 7TH ST,NaN,0.0000,0.0000


In [466]:
df1 = dataset_624['DATE OCC'].dt.date.value_counts().sort_index().reset_index()
df1.columns = ['DATE','Count']

df1['DATE'] = pd.to_datetime(df1['DATE']) - pd.to_timedelta(7, unit='d')
df1= df1.groupby([pd.Grouper(key='DATE', freq='W')])['Count'].sum()

df1 = pd.DataFrame(data=df1.values, columns=['Count'])
df1 = df1.values
df1 = df1.astype('int32')
df1
# df1 = dataset_510['DATE OCC'].dt.date.value_counts().sort_index().reset_index()
# df1.columns = ['DATE','Count']
# # df1.plot(x='DATE',y='Count')
# dataset_510.groupby([pd.Grouper(key='DATE OCC', freq='W')]).count()

array([[266],
       [335],
       [332],
       [351],
       [360],
       [354],
       [353],
       [328],
       [350],
       [383],
       [314],
       [288],
       [247],
       [260],
       [229],
       [277],
       [303],
       [332],
       [313],
       [273],
       [325],
       [267],
       [276],
       [322],
       [362],
       [345],
       [329],
       [297],
       [344],
       [360],
       [340],
       [329],
       [330],
       [366],
       [325],
       [313],
       [302],
       [332],
       [304],
       [324],
       [291],
       [314],
       [308],
       [268],
       [283],
       [294],
       [302],
       [275],
       [274],
       [277],
       [286],
       [245],
       [254],
       [241],
       [260],
       [279],
       [243],
       [285],
       [268],
       [279],
       [268],
       [288],
       [243],
       [269],
       [292],
       [313],
       [276],
       [258],
       [270],
       [283],
       [293],
      

In [467]:
list_graphs = []
graph_1 = go.Scatter(x=list(range(0,len(df1))),y=df1.flatten(),name="Dataset")
list_graphs.append(graph_1)
average = moving_average(df1)
graph_2 = go.Scatter(x=list(range(0,len(df1))),y=average,name="Average")
list_graphs.append(graph_2)
fig = go.Figure(data=list_graphs)
fig.update_layout(width=1024,height=500,title="Dataset of crimes in ",xaxis=dict(title='Weeks'),yaxis=dict(title='Number of crimes'))

fig.show()

In [468]:

scaler = MinMaxScaler()
df1 = scaler.fit_transform(df1)

In [469]:
# dataset_510['DATE OCC'] = pd.to_datetime(dataset_510['DATE OCC']) - pd.to_timedelta(7, unit='d')

df1
train_size = int(len(df1) * 0.7)
print(train_size)
test_size = len(df1) - train_size
print(test_size)
train, test = df1[0:train_size,:], df1[train_size:len(df1),:]

# df1 = df1.drop(['DATE'], axis=1, index=None, columns=None, level=None, inplace=False, errors='raise')\

60
26


In [470]:
# dataset_510['DATE OCC'] = pd.to_datetime(dataset_510['DATE OCC']) - pd.to_timedelta(7, unit='d')
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# df1 = df1.drop(['DATE'], axis=1, index=None, columns=None, level=None, inplace=False, errors='raise')\

In [471]:
trainX.shape, trainY.shape, testX.shape, testY.shape

((59, 1), (59,), (25, 1), (25,))

In [472]:
trainX = np.reshape(trainX, (trainX.shape[0],1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0],1, testX.shape[1]))
# trainX.shape

In [473]:

from keras.models import Sequential
from keras.layers import Dense, LSTM



model = Sequential()
model.add(LSTM(8,activation='tanh',return_sequences=True, input_shape=(1,look_back)))
model.add(LSTM(4,activation='tanh',return_sequences=False, input_shape=(1,look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.fit(trainX, trainY, epochs=500, batch_size=4, verbose=1)

Epoch 1/500
15/15 [==============================] - 4s 4ms/step - loss: 0.5863
Epoch 2/500
15/15 [==============================] - 0s 3ms/step - loss: 0.5231
Epoch 3/500
15/15 [==============================] - 0s 2ms/step - loss: 0.4680
Epoch 4/500
15/15 [==============================] - 0s 2ms/step - loss: 0.4125
Epoch 5/500
15/15 [==============================] - 0s 2ms/step - loss: 0.3564
Epoch 6/500
15/15 [==============================] - 0s 2ms/step - loss: 0.2999
Epoch 7/500
15/15 [==============================] - 0s 4ms/step - loss: 0.2442
Epoch 8/500
15/15 [==============================] - 0s 7ms/step - loss: 0.1901
Epoch 9/500
15/15 [==============================] - 0s 4ms/step - loss: 0.1393
Epoch 10/500
15/15 [==============================] - 0s 2ms/step - loss: 0.0943
Epoch 11/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0567
Epoch 12/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0300
Epoch 13/500
15/15 [=================

In [474]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

1/1 [==============================] - 0s 13ms/step


In [475]:
trainPredict.shape, trainY.shape
# trainPredict
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [476]:
trainY[0],trainPredict[:,0]

(array([335., 332., 351., 360., 354., 353., 328., 350., 383., 314., 288.,
        247., 260., 229., 277., 303., 332., 313., 273., 325., 267., 276.,
        322., 362., 345., 329., 297., 344., 360., 340., 329., 330., 366.,
        325., 313., 302., 332., 304., 324., 291., 314., 308., 268., 283.,
        294., 302., 275., 274., 277., 286., 245., 254., 241., 260., 279.,
        243., 285., 268., 279.]),
 array([282.26245, 328.69577, 326.65457, 339.59534, 345.731  , 341.64047,
        340.95874, 323.9347 , 338.9137 , 361.3985 , 314.43503, 296.91287,
        269.7826 , 278.3024 , 258.1389 , 289.5638 , 306.99933, 326.65457,
        313.7579 , 286.90292, 321.89624, 282.92407, 288.89798, 319.85922,
        347.09445, 335.50604, 324.61444, 302.95663, 334.82468, 345.731  ,
        332.09982, 324.61444, 325.29434, 349.8211 , 321.89624, 313.7579 ,
        306.32486, 326.65457, 307.6741 , 321.21704, 298.9246 , 314.43503,
        310.3756 , 283.58615, 293.56686, 300.93918, 306.32486, 288.2325 ,
    

In [477]:
testY[0],testPredict[:,0]

(array([288., 243., 269., 292., 313., 276., 258., 270., 283., 293., 296.,
        312., 307., 314., 342., 338., 312., 387., 353., 357., 353., 338.,
        338., 288.,  11.]),
 array([283.58615, 296.91287, 267.17905, 284.24866, 299.5958 , 313.7579 ,
        288.89798, 276.98615, 284.91156, 293.56686, 300.2673 , 302.28387,
        313.08102, 309.69986, 314.43503, 333.46213, 330.73795, 313.08102,
        364.1184 , 340.95874, 343.68573, 340.95874, 330.73795, 330.73795,
        296.91287], dtype=float32))

In [478]:
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 27.26 RMSE
Test Score: 62.12 RMSE


In [479]:
trainPredictPlot = np.empty_like(df1)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

In [480]:
testPredictPlot = np.empty_like(df1)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back):len(df1)-1, :] = testPredict

In [486]:
listGraph = []
listGraph.append(go.Scatter(x=list(range(0,len(df1))),y=scaler.inverse_transform(df1).flatten(),name="Original dataset"))
listGraph.append(go.Scatter(x=list(range(0,len(trainPredictPlot))),y=trainPredictPlot.flatten(),name="Predict with Train set"))
listGraph.append(go.Scatter(x=list(range(0,len(testPredictPlot))),y=testPredictPlot.flatten(),name="Predict with Test set"))

fig = go.Figure(data=listGraph)
fig.update_layout(width=1024,height=500,title="[LSTM] Prediction for Battery - Simple Assault (Code 624)",xaxis=dict(title='Weeks'),yaxis=dict(title='Number of crimes'))

fig.show()